In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import corvinus_tardis_data as ctd
import tardis_dev
# this is a special fix for tardis_dev not liking running from a notebook. it prefers running as a script
import nest_asyncio
nest_asyncio.apply()

In [10]:
tardis_dev.datasets.download_async

AttributeError: module 'tardis_dev.datasets' has no attribute 'download_async'

In [ ]:
ctd.example_download_of_a_dataset()